## Synthetic Baseline
Running Basecalling using Dorado and checking accuracy metrics for Motif Detection using the Synthetic Dataset created

### Setup

In [1]:
import pandas as pd

import os

# Changing directory to where everything is stored
os.chdir(r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code")

dataset = pd.read_pickle("short_read.pkl")
dataset.head()

,read_id,base_seq,motif_seq,squiggle
0,S1_1!seq_3789!441!1000!+,TACCGAATCTGGTAAGCTAAAGTCGTCTCGTTGCATCGAAAGTCGT...,448032759800276471051389727,"[525, 520, 529, 532, 526, 521, 525, 518, 526, ..."
1,S1_10!seq_3785!180!1000!-,AGTCTAGCTCGTACGTTCGAGACTTGCAATCGCTAAGCTAAAGTCG...,2414234129277705527229226850840105263073,"[456, 454, 465, 454, 473, 450, 456, 452, 512, ..."
2,S1_100!seq_3789!630!1000!+,TTGAAGCTGAATTCGTCGAACATGCTTCCAAGTCTAGCTCGTACGT...,800276471051389727,"[497, 502, 491, 497, 480, 494, 478, 498, 493, ..."
3,S1_1000!seq_3782!679!1000!+,CATTCGTCGAACATGCTTCCAAGTCTAGCTCGTACGTTCGAAAGTC...,8049127409240628,"[495, 501, 492, 493, 479, 509, 482, 520, 519, ..."
4,S1_1001!seq_3784!429!1000!+,CAAGCTTAGACAGTCTAGCTCGTACGTTCGAGCTACTCGACTTGAA...,0935479054849128404617924203,"[443, 440, 443, 438, 446, 443, 446, 449, 441, ..."


In [2]:
motif_choices = [
    "AGTCTAGCTCGTACGTTCGA",
    "TGGCACTCATCAATCCGTAT",
    "GACTTGCAATCGCTAAGCTA",
    "CTATGAGGTCTGCCTTACAC",
    "AAGTCGTCTCGTTGCATCGA",
    "TCCAGTAGATCGTCAGCTTC",
    "GTACCGAATCTGGTAAGCTA",
    "CGATCATCGCAAGCTTAGAC",
    "ATTCGTCGAACATGCTTCCA",
    "GCTACTCGACTTGAAGCTGA"
]

### Basecalled file

I wonder what the formatting is going to be, but it may have sampled from the squiggle. If it is a direct translation, then it really helps me out, we can compare at base level, whole motif reconstruction, and with an appropiate motif search algorithm (basically, given the information, can we create a motif algorithm to determine the motif at that position

In [3]:
reads = pd.read_pickle("basecalled_reads_short_read.pkl")

In [4]:
reads.head()

,read_id,read
0,S1_1!seq_3789!441!1000!+,AATCTGGTAAGCTAAAGTCGTCTCGTTGCATCGAAAGTCGTCTCGT...
1,S1_100!seq_3789!630!1000!+,AGCTGAATTCGTCGAACATGCTTCAAGTCTAGCTCGTACGTTCGAA...
2,S1_1001!seq_3784!429!1000!+,GCTTAGACAGTCTAGCTCGTACGTTCCCGAGCTACTCGACTTGAAG...
3,S1_1003!seq_3789!639!1000!+,CGTCGAACATGCTTCAAGTCTAGCTCGTACGTTCGAAGTCTAGCTC...
4,S1_1005!seq_3788!234!1000!-,GCATGTTCGACGAATTCGAACGTACGAACTAGACTAGAACGTGAGC...


In [5]:
dataset_read_ids = dataset['read_id'].to_numpy().tolist()
dataset_bases = dataset['base_seq'].to_numpy().tolist()
dataset_motifs = dataset['motif_seq'].to_numpy().tolist()

In [6]:
dataset_dict = {}
for i in range(len(dataset_read_ids)):
    dataset_dict[dataset_read_ids[i]] = (dataset_bases[i], dataset_motifs[i])

In [7]:
reads_read_ids = reads['read_id'].to_numpy().tolist()
reads_bases = reads['read'].to_numpy().tolist()

In [8]:
reads_dict = {}
for i in range(len(reads_read_ids)):
    reads_dict[reads_read_ids[i]] = reads_bases[i]

In [9]:
base_error_rate = []
for read_id in reads_read_ids[:5]:
    dataset_bases = dataset_dict[read_id][0]
    reads_bases = reads_dict[read_id]
    n_bases = 0 
    correct_bases = 0
    extras = 0
    for base_ptr in range(len(reads_bases)):
        n_bases += 1 
        try:
            if dataset_bases[base_ptr] == reads_bases[base_ptr]:
                correct_bases+=1
        except:
            extras+=1
    print(correct_bases)
    print(n_bases)
    base_level_acc = correct_bases/n_bases
    print(base_level_acc)
    base_error_rate.append(base_level_acc)
    

144
552
0.2608695652173913
61
359
0.16991643454038996
119
557
0.21364452423698385
83
343
0.24198250728862974
183
743
0.24629878869448182


In [10]:

read_id = reads_read_ids[8]

def count_freq(basepair_str):
    count_dict = {'A':0,'C':0, 'T':0, 'G':0}
    for base in basepair_str:
        count_dict[base] += 1
    return count_dict

# Checking if aligning will help
dataset_bases = dataset_dict[read_id][0]

count_dict_dataset = count_freq(dataset_bases)

In [11]:
count_dict_dataset

{'A': 193, 'C': 209, 'T': 213, 'G': 169}

In [12]:
reads_bases = reads_dict[read_id]
reads_bases[:10]
read_dict_dataset = count_freq(reads_bases)
read_dict_dataset

{'A': 188, 'C': 204, 'T': 210, 'G': 169}

In [14]:
for i in read_dict_dataset.keys():
    print(i)

A
C
T
G


In [22]:
import math

def measure_count_error(count_dict, read_dict):
    rel_error = 0
    for key in read_dict.keys():
        rel_error += abs(count_dict[key] - read_dict[key])/max(count_dict[key], read_dict[key])

    return rel_error/4

measure_count_error(count_dict_dataset, read_dict_dataset)

0.01597867194240635

In [37]:

count_errs = []

for read_id in reads_read_ids:
    dataset_bases = dataset_dict[read_id][0]
    reads_bases = reads_dict[read_id]
    
    count_dict_dataset = count_freq(dataset_bases)
    read_dict_dataset = count_freq(reads_bases)

    count_error = measure_count_error(count_dict_dataset, read_dict_dataset)
    count_errs.append(count_error)

print("Error Rate")
print(f"{sum(count_errs)/len(count_errs) * 100}%")

Error Rate
9.878085557422496%


In [36]:
print("Accuracy in Counts")
print(f"{100- (sum(count_errs)/len(count_errs) * 100)}%")

Accuracy in Counts
90.1219144425775%
